In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.696325
Training accuracy: 10.6%
Validation accuracy: 15.1%
Loss at step 100: 2.348341
Training accuracy: 71.2%
Validation accuracy: 69.7%
Loss at step 200: 1.859651
Training accuracy: 74.3%
Validation accuracy: 72.2%
Loss at step 300: 1.603100
Training accuracy: 75.9%
Validation accuracy: 73.5%
Loss at step 400: 1.436622
Training accuracy: 77.0%
Validation accuracy: 74.0%
Loss at step 500: 1.315916
Training accuracy: 77.8%
Validation accuracy: 74.3%
Loss at step 600: 1.222595
Training accuracy: 78.3%
Validation accuracy: 74.7%
Loss at step 700: 1.147147
Training accuracy: 79.0%
Validation accuracy: 74.9%
Loss at step 800: 1.084247
Training accuracy: 79.5%
Validation accuracy: 75.0%
Test accuracy: 82.5%


In [13]:
#SGD
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #     if (step % 500 == 0):
    #         print("aaa %d" %train_labels.shape[0])
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.246441
Minibatch accuracy: 7.8%
Validation accuracy: 11.0%
Minibatch loss at step 500: 1.195624
Minibatch accuracy: 79.7%
Validation accuracy: 75.3%
Minibatch loss at step 1000: 1.562811
Minibatch accuracy: 75.8%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 0.892502
Minibatch accuracy: 78.9%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 1.020694
Minibatch accuracy: 82.8%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 1.013018
Minibatch accuracy: 74.2%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 0.925958
Minibatch accuracy: 76.6%
Validation accuracy: 78.6%
Test accuracy: 86.5%


In [17]:
#Problem: 1 hidden layer; graph building
batch_size = 128
depth = 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
      
    #layer 1
    # Variables.
    weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, depth])) #784*1024
    biases1 = tf.Variable(tf.zeros([depth]))
  
    # Training computation.
    logits1 = tf.matmul(tf_train_dataset, weights1) + biases1   #20000x784 x 784x1024 = 20000x1024
    
    #hidden layer
    h_fc1 = tf.nn.relu(logits1)
    
    #layer2
    ## Variables.
    weights2 = tf.Variable(
        tf.truncated_normal([depth, num_labels])) 
    biases2 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits2 = tf.matmul(h_fc1, weights2) + biases2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits2)
    #validation
    v_fc1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
        
    valid_prediction = tf.nn.softmax(
        tf.matmul(v_fc1, weights2) + biases2)
    #test
    t_fc1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.softmax(tf.matmul(t_fc1, weights2) + biases2)

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 328.028992
Minibatch accuracy: 14.1%
Validation accuracy: 27.5%
Minibatch loss at step 500: 13.643013
Minibatch accuracy: 82.8%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 12.100320
Minibatch accuracy: 80.5%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 6.894794
Minibatch accuracy: 86.7%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 2.681479
Minibatch accuracy: 86.7%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 8.130533
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 3.232700
Minibatch accuracy: 78.9%
Validation accuracy: 82.3%
Test accuracy: 89.8%


In [15]:
#multi hidden layers
from tensorflow.python.ops import init_ops
batch_size=128
hidden_layers = [1024, 1024, 1024, 10]
dnn_graph = tf.Graph()
with dnn_graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    keep_prob = tf.placeholder("float")
    
    #tensor_in = tf_train_dataset
    
    relu_out = tf_train_dataset
    for i, n_hidden in enumerate(hidden_layers):
        v_name = "layer%d" %i
        with tf.variable_scope(v_name):
            total_arg_size = relu_out.get_shape().as_list()[1]
            matrix = tf.get_variable("Matrix", [total_arg_size, n_hidden])
            #w = tf.get_variable("w", shape=[image_size * image_size, image_size * image_size], dtype=tf.float32, initializer=tf.truncated_normal_initializer())
            b = tf.get_variable("b", shape=[n_hidden], dtype=tf.float32, initializer=init_ops.constant_initializer(0.0))
            relu_out = tf.nn.relu(tf.matmul(relu_out, matrix) + b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(relu_out, tf_train_labels))
    optimizer = tf.train.GradientDescentOptimizer(0.005).minimize(loss)
    train_prediction = tf.nn.softmax(relu_out)
    
    #test sets
    t_relu_out = tf_test_dataset
    for i, n_hidden in enumerate(hidden_layers):
        v_name = "layer%d" %i
        with tf.variable_scope(v_name, reuse=True):
            t_total_arg_size = t_relu_out.get_shape().as_list()[1]
            t_matrix = tf.get_variable("Matrix")
            t_b = tf.get_variable("b", shape=[n_hidden], dtype=tf.float32)
            t_relu_out = tf.nn.relu(tf.matmul(t_relu_out, t_matrix) + t_b)
    test_prediction = tf.nn.softmax(t_relu_out) 
    
    #valid sets
    v_relu_out = tf_valid_dataset
    for i, n_hidden in enumerate(hidden_layers):
        v_name = "layer%d" %i
        with tf.variable_scope(v_name, reuse=True):
            v_total_arg_size = t_relu_out.get_shape().as_list()[1]
            v_matrix = tf.get_variable("Matrix")
            v_b = tf.get_variable("b", shape=[n_hidden], dtype=tf.float32)
            v_relu_out = tf.nn.relu(tf.matmul(v_relu_out, v_matrix) + v_b)
    valid_prediction = tf.nn.softmax(v_relu_out)

In [16]:
num_steps = 3001

with tf.Session(graph=dnn_graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels} 
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.324198
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Test accuracy: 9.9%
Minibatch loss at step 500: 1.086033
Minibatch accuracy: 72.7%
Validation accuracy: 70.7%
Test accuracy: 77.5%
Minibatch loss at step 1000: 0.921667
Minibatch accuracy: 71.1%
Validation accuracy: 73.5%
Test accuracy: 79.7%
Minibatch loss at step 1500: 0.492810
Minibatch accuracy: 85.9%
Validation accuracy: 81.0%
Test accuracy: 88.1%
Minibatch loss at step 2000: 0.482086
Minibatch accuracy: 88.3%
Validation accuracy: 82.0%
Test accuracy: 88.8%
Minibatch loss at step 2500: 0.592105
Minibatch accuracy: 82.8%
Validation accuracy: 82.7%
Test accuracy: 89.4%
Minibatch loss at step 3000: 0.594670
Minibatch accuracy: 82.8%
Validation accuracy: 83.0%
Test accuracy: 89.9%
